In [2]:
!pip install langchain langchain-groq chromadb langchain-community google.colab

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_groq import ChatGroq
from google.colab import userdata
groq_api_key=userdata.get('GROQ_API_KEY')
llm=ChatGroq(model_name="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
llm

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://www.accenture.com/in-en/careers/jobdetails?id=ATCI-R1-S1752597_en&title=Application%20Developer#accordion-457aff5aa2-item-959cb67f76")
page_content=loader.load().pop().page_content
page_content

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE
    {page_data}
    ###INSTRUCTION
    the scraped text is from a career page in a website
    Your job is to extract the job postings and return them in json format containing following keys:
    role,experience,skills and description
    Only return the valid JSON
    ###VALID JSON(NO PREAMBLE)
    """
)
chain_extract=prompt_extract|llm
res=chain_extract.invoke(input={"page_data":page_content})
res.content

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
json_parser=JsonOutputParser()
res=json_parser.parse(res.content)
res

In [ ]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

In [ ]:
import chromadb
import uuid
client=chromadb.PersistentClient('vectorstore')
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
  for _,row in df.iterrows():
    collection.add(documents=row["Techstack"],
                   metadatas={"links":row["Links"]},
                   ids=[str(uuid.uuid4())])

In [ ]:
links=collection.query(query_texts=["Experience in Python","Expertise in React"],n_results=2).get("metadatas",[])
print(links)
#har skill ke 2 results

In [ ]:
job=res
job["skills"]

In [ ]:
links=collection.query(query_texts=job["skills"],n_results=2).get("metadatas",[])
print(links)
#har skill ke 2 results

In [ ]:
prompt_email=PromptTemplate.from_template(
            """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
)
chain_email=prompt_email | llm
res=chain_email.invoke({"job_description":str(job),"link_list":links})
print(res.content)